## Let's see some code - Interacting with the environment

The code snippet below shows how to initialise the environment and step through in an 'offline' manner
(Here offline means that the environment is generating some recommendations for us).
We print out the results from the environment at each step.

In [15]:
import gym, recogym

# env_0_args is a dictionary of default parameters (i.e. number of products)
from recogym import env_1_args, Configuration

# You can overwrite environment arguments here:
env_1_args['random_seed'] = 42

# Initialize the gym for the first time by calling .make() and .init_gym()
env = gym.make('reco-gym-v1')
env.init_gym(env_1_args)

# .reset() env before each episode (one episode per user).
env.reset()
done = False

# Counting how many steps.
i = 0

observation, reward, done = None, 0, False
while not done:
    action, observation, reward, done, info = env.step_offline(observation, reward, done)
    print(f"STEP: {i} - ACTION: {action} - OBSERVATION: {observation.sessions()} - REWARD: {reward}")
    i += 1

STEP: 0 - ACTION: None - OBSERVATION: [{'t': 0, 'u': 0, 'z': 'pageview', 'v': 0}] - REWARD: None
STEP: 1 - ACTION: {'t': 1, 'u': 0, 'a': 3, 'ps': 0.1, 'ps-a': ()} - OBSERVATION: [] - REWARD: 0
STEP: 2 - ACTION: {'t': 2, 'u': 0, 'a': 4, 'ps': 0.1, 'ps-a': ()} - OBSERVATION: [] - REWARD: 0
STEP: 3 - ACTION: {'t': 3, 'u': 0, 'a': 5, 'ps': 0.1, 'ps-a': ()} - OBSERVATION: [] - REWARD: 0


Okay, there's quite a bit going on here:
- `action`, is a number between `0` and `num_products - 1` that references the index of the product recommended.
- `observation` will either be `None` or a session of Organic data, showing the index of products the user views.
- `reward` is 0 if the user does not click on the recommended product and 1 if they do. Notice that when a user clicks on a product (Wherever the reward is 1), they start a new Organic session.
- `done` is a True/False flag indicating if the episode (aka user's timeline) is over.
- `info` currently not used, so it is always an empty dictionary.

Also, notice that the first `action` is `None`.  In our implementation, the agent observes Organic behaviour before
recommending anything.

Now, we will show calling the environment in an online manner, where the agent needs to supply an action.
For demonstration purposes, we will create a list of hard-coded actions.

In [16]:
# Create list of hard coded actions.
actions = [None] + [1, 2, 3, 4, 5, 6, 7, 8]

# Reset env and set done to False.
env.reset()
done = False

# Counting how many steps.
i = 0

while not done and i < len(actions):
    action = actions[i]
    observation, reward, done, info = env.step(action)
    print(f"Step: {i} - Action: {action} - Observation: {observation.sessions()} - Reward: {reward}")
    i += 1

Step: 0 - Action: None - Observation: [{'t': 0, 'u': 0, 'z': 'pageview', 'v': 1}] - Reward: None
Step: 1 - Action: 1 - Observation: [] - Reward: 0
Step: 2 - Action: 2 - Observation: [] - Reward: 0
Step: 3 - Action: 3 - Observation: [] - Reward: 0
Step: 4 - Action: 4 - Observation: [] - Reward: 0
Step: 5 - Action: 5 - Observation: [{'t': 6, 'u': 0, 'z': 'pageview', 'v': 4}, {'t': 7, 'u': 0, 'z': 'pageview', 'v': 4}, {'t': 8, 'u': 0, 'z': 'pageview', 'v': 4}, {'t': 9, 'u': 0, 'z': 'pageview', 'v': 4}, {'t': 10, 'u': 0, 'z': 'pageview', 'v': 4}, {'t': 11, 'u': 0, 'z': 'pageview', 'v': 6}, {'t': 12, 'u': 0, 'z': 'pageview', 'v': 4}, {'t': 13, 'u': 0, 'z': 'pageview', 'v': 4}, {'t': 14, 'u': 0, 'z': 'pageview', 'v': 4}, {'t': 15, 'u': 0, 'z': 'pageview', 'v': 6}, {'t': 16, 'u': 0, 'z': 'pageview', 'v': 4}, {'t': 17, 'u': 0, 'z': 'pageview', 'v': 1}, {'t': 18, 'u': 0, 'z': 'pageview', 'v': 4}, {'t': 19, 'u': 0, 'z': 'pageview', 'v': 4}, {'t': 20, 'u': 0, 'z': 'pageview', 'v': 4}, {'t': 21, '

## Creating our Hello World Agent

Now that we see have seen how the offline and online versions of the environment work,
it is time to code our first recommendation agent!

Technically, an agent can be anything that produces actions for the environment to use.
However, we will show you the object-oriented way we like to create agents.

Below is the code for a very simple agent
The agent records merely how many times a user sees each product organically,
then when required to make a recommendation, the agent chooses a product randomly in proportion with
a number of times the user has viewed it.

In [17]:
import numpy as np
from numpy.random import choice
from recogym.agents import Agent

# Define an Agent class.
class HelloWorldAgent(Agent):

    def __init__(self, config):
        
        super(HelloWorldAgent, self).__init__(config) # Set number of products as an attribute of the Agent.

        self.organic_views = np.zeros(self.config.num_products) # Track number of times each item viewed in Organic session.

    def train(self, observation, action, reward, done):

        # Train method learns from a tuple of data. This method can be called for offline or online learning
        # Adding organic session to organic view counts.

        if observation:
            for session in observation.sessions():
                self.organic_views[session['v']] += 1

    def act(self, observation, reward, done):

        # Act method returns an action - based on current observation and past history
        # Choosing action randomly in proportion with number of views.

        prob = self.organic_views / sum(self.organic_views)

        action = choice(self.config.num_products, p = prob) # GENERATE ONLINE ACTION

        return {
            **super().act(observation, reward, done),
            **{
                'a': action,
                'ps': prob[action]
            }
        }

The `HelloWorldAgent` class above demonstrates our preferred way to create agents for RecoGym.

Notice how we have both a `train` and `act` method present. 

The `train` method is designed to take in training data from the environments `step_offline` 
method and thus has nothing to return, while the `act` method must return an action to pass 
back into the environment.

The code below highlights how one would use this agent for first offline training and then using the learned
knowledge to make recommendations online.

In [18]:
# Instantiate instance of HelloWorldAgent class.
num_products = 10
agent = HelloWorldAgent(Configuration({
    **env_1_args,
    'num_products': num_products,
}))

# Resets random seed back to 42, or whatever we set it to in env_0_args.
env.reset_random_seed()

# Train on 1000 users offline.
num_offline_users = 1000

for _ in range(num_offline_users):

    # Reset env and set done to False.
    env.reset()
    done = False

    observation, reward, done = None, 0, False
    while not done:
        old_observation = observation
        action, observation, reward, done, info = env.step_offline(observation, reward, done)
        agent.train(old_observation, action, reward, done) # TRAIN OFFLINE

# Train on 100 users online and track click through rate.
num_online_users = 100
num_clicks, num_events = 0, 0

for _ in range(num_online_users):

    # Reset env and set done to False.
    env.reset()
    observation, _, done, _ = env.step(None)
    reward = None
    done = None

    while not done: # ----- LOOP

        action = agent.act(observation, reward, done) # create recommendation

        observation, reward, done, info = env.step(action['a'])

        # Used for calculating click through rate.
        num_clicks += 1 if reward == 1 and reward is not None else 0
        num_events += 1

ctr = num_clicks / num_events

print(f"Click Through Rate: {ctr:.4f}")

Click Through Rate: 0.0132


## Testing our hello world agent

Now we have created our hello world agent, and we should test it against an even simpler baseline -
one that performs no learning and recommends products uniformly at random.
To do this, we will first load a more complex version of the toy data environment called `reco-gym-v1`.

Next, we will load another agent for our agent to compete against each other.
Here you can see we make use of the `RandomAgent` and create an instance of it in addition to our `HelloWorldAgent`.

In [19]:
import gym, recogym
from recogym import env_1_args

from copy import deepcopy

env_1_args['random_seed'] = 42

env = gym.make('reco-gym-v1')
env.init_gym(env_1_args)

# Import the random agent.
from recogym.agents import RandomAgent, random_args

# Create the two agents.
num_products = env_1_args['num_products']

agent_hello_world = HelloWorldAgent(Configuration(env_1_args))

agent_random = RandomAgent(Configuration({
    **env_1_args,
    **random_args,
}))

Now we have instances of our two agents. We can use the `test_agent` method from RecoGym and compare there performance.

To use `test_agent`, one must provide a copy of the current env, a copy of the agent class, the number of training users and the number of testing users. 

In [20]:
# Credible interval of the CTR median and 0.025 0.975 quantile.

recogym.test_agent(deepcopy(env), deepcopy(agent_random), 1000, 1000)

Organic Users: 0it [00:00, ?it/s]
Users: 100%|██████████| 1000/1000 [00:14<00:00, 68.94it/s]
Organic Users: 0it [00:00, ?it/s]
Users: 100%|██████████| 1000/1000 [00:15<00:00, 64.46it/s]


START: Agent Training #0
START: Agent Training @ Epoch #0
END: Agent Training @ Epoch #0 (14.512444972991943s)
START: Agent Evaluating @ Epoch #0
END: Agent Evaluating @ Epoch #0 (15.659083843231201s)


(0.011463611074356707, 0.010743287419571399, 0.012215054884766707)

In [21]:
# Credible interval of the CTR median and 0.025 0.975 quantile.

recogym.test_agent(deepcopy(env), deepcopy(agent_hello_world), 1000, 1000)

Organic Users: 0it [00:00, ?it/s]
Users: 100%|██████████| 1000/1000 [00:14<00:00, 68.88it/s]
Organic Users: 0it [00:00, ?it/s]
Users: 100%|██████████| 1000/1000 [00:17<00:00, 56.36it/s]


START: Agent Training #0
START: Agent Training @ Epoch #0
END: Agent Training @ Epoch #0 (14.523025751113892s)
START: Agent Evaluating @ Epoch #0
END: Agent Evaluating @ Epoch #0 (17.89901900291443s)


(0.01419748696611417, 0.013389659432901099, 0.015036695334252781)

We see an improvement in the click-through rate for the hello world agent.